In [110]:
import pandas as pd
import glob
import os
from tqdm import tqdm_notebook as tqdm
#preprocessing
dirlist = ["dokujo-tsushin","it-life-hack","kaden-channel","livedoor-homme",
           "movie-enter","peachy","smax","sports-watch","topic-news"]
df = pd.DataFrame(columns=["label","class","news"])
for index , i in enumerate(tqdm(dirlist)):
    path = "/home/jovyan/work/livedoor/text/"+i+"/*.txt" 
    files = glob.glob(path)
    files.pop()
    for j in tqdm(files):
        f = open(j,encoding="utf-8")
        data = f.read() 
        f.close()
        t = pd.Series(['__label__'+str(index),i,"".join(data.split("\n")[3:])],index = df.columns)
        df  = df.append(t,ignore_index=True)

In [111]:
import logging
import numpy as np
from gensim.models import Word2Vec
import MeCab
import time
from sklearn.preprocessing import normalize
import sys
import re

start = time.time()
tokenizer =  MeCab.Tagger("-Owakati")  
sentences = []
hisaka = []
print ("Parsing sentences from training set...")

# Loop over each news article.
for review in tqdm(df["news"]):
    try:
        # Split a review into parsed sentences.
        result = tokenizer.parse(review).replace("\u3000","").replace("\n","")
        result = re.sub(r'[0123456789０１２３４５６７８９！＠＃＄％＾＆\-|\\＊\“（）＿■×※⇒—●(：〜＋=)／*&^%$#@!~`){}…\[\]\"\'\”:;<>?＜＞？、。・,./『』【】「」→←○]+', "", result)
        h = result.split(" ")
        h = list(filter(("").__ne__, h))
        sentences.append(result)
    except:
        continue

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

Parsing sentences from training set...


In [150]:

df['news'] = sentences
df.to_csv("/home/jovyan/work/livedoor_fastText.csv",columns=['label', 'news'],index=False,header=False )

In [166]:
from gensim.models import FastText
f = open("/home/jovyan/work/livedoor_fastText.csv")
data1 = f.read()  # ファイル終端まで全て読んだデータを返す
f.close()

model = FastText(data1, size=4, window=3, min_count=1, iter=10)

2018-11-19 08:21:10,308 : INFO : collecting all words and their counts
2018-11-19 08:21:10,309 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2018-11-19 08:21:10,310 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-11-19 08:21:10,321 : INFO : PROGRESS: at sentence #10000, processed 10000 words, keeping 592 word types
2018-11-19 08:21:10,332 : INFO : PROGRESS: at sentence #20000, processed 20000 words, keeping 894 word types
2018-11-19 08:21:10,341 : INFO : PROGRESS: at sentence #30000, processed 30000 words, keeping 1090 word types
2018-11-19 08:21:10,350 : INFO : PROGRESS: at sentence #40000, processed 40000 words, keeping 1257 word types
2018-11-19 08:21:10,361 : INFO : PROGRESS: at sentence #50000, processed 50000 words, keeping 1334 word types
2018-11-19 08:21:10,374 : INFO : PROGRESS: at sentence #60000, processed 60000 words, keeping 1437 word types
2018-11-19 08

In [167]:
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("/home/jovyan/work/fasttext.model")

In [168]:
model.save(fname)

2018-11-19 08:27:28,380 : INFO : saving FastText object under /home/jovyan/work/fasttext.model, separately None
2018-11-19 08:27:28,382 : INFO : not storing attribute vectors_norm
2018-11-19 08:27:28,383 : INFO : not storing attribute vectors_vocab_norm
2018-11-19 08:27:28,384 : INFO : not storing attribute vectors_ngrams_norm
2018-11-19 08:27:28,388 : INFO : not storing attribute buckets_word
2018-11-19 08:27:28,410 : INFO : saved /home/jovyan/work/fasttext.model


In [169]:
model = FastText.load(fname)

2018-11-19 08:27:29,837 : INFO : loading FastText object from /home/jovyan/work/fasttext.model
2018-11-19 08:27:29,860 : INFO : loading wv recursively from /home/jovyan/work/fasttext.model.wv.* with mmap=None
2018-11-19 08:27:29,861 : INFO : setting ignored attribute vectors_norm to None
2018-11-19 08:27:29,862 : INFO : setting ignored attribute vectors_vocab_norm to None
2018-11-19 08:27:29,863 : INFO : setting ignored attribute vectors_ngrams_norm to None
2018-11-19 08:27:29,864 : INFO : setting ignored attribute buckets_word to None
2018-11-19 08:27:29,865 : INFO : loading vocabulary recursively from /home/jovyan/work/fasttext.model.vocabulary.* with mmap=None
2018-11-19 08:27:29,866 : INFO : loading trainables recursively from /home/jovyan/work/fasttext.model.trainables.* with mmap=None
2018-11-19 08:27:29,867 : INFO : loaded /home/jovyan/work/fasttext.model


In [170]:
existent_word = '女'
existent_word in model.wv.vocab

True

In [175]:
similarities = model.wv.most_similar(positive=['人', '男'], negative=['女'])

In [176]:
similarities

[('符', 0.9990203976631165),
 ('買', 0.9963841438293457),
 ('眩', 0.9887847900390625),
 ('벌', 0.988580048084259),
 ('残', 0.9874111413955688),
 ('薔', 0.9821394681930542),
 ('藩', 0.9819920063018799),
 ('除', 0.9770439863204956),
 ('嘲', 0.9758253693580627),
 ('揖', 0.9748659133911133)]